In [ ]:
from bart_score import BARTScorer
import re
import numpy as np
import json
from sentence_transformers import SentenceTransformer, util

In [ ]:
bart_scorer = BARTScorer(device='cuda:0', checkpoint="/data/nlp_dev/BARTScore-main/bart_large_cnn/")#checkpoint='facebook/bart-large-cnn') 
bart_scorer.load(path='/data/nlp_dev/BARTScore-main/bart_score.pth')

In [ ]:
gen_file = "saveGenMSRVTT-uniformsample-diverse.json" 
fcc_file = "MSRVTT-Order.json" 

with open(fcc_file, 'r') as fcc_file:
    fcc_data = json.load(fcc_file)
    print(fcc_data)

with open(gen_file, 'r') as gen_file:
    gen_data = json.load(gen_file)
    print(gen_data)

correct = 0
num = 0
for item in fcc_data:
    num = num + 1
    video_id = item['video_id']
    for idx in gen_data:
        if idx['video_folder_name'] == video_id:
            describe_content = idx['describe_content']
    
    result_list = re.split(r'[.]', describe_content) 
    if '</s>' in result_list[-1] or '' in result_list[-1]:
        result_list.pop()
    llava_txt_num = len(result_list)
    
    bart_score = []
    for c_option in item['captions']: #5 sentence
        for llava_result in result_list:
            bart_score_tmp = bart_scorer.score([llava_result],[c_option])
            bart_score.append(bart_score_tmp[0])

    first_max = np.mean(bart_score[:llava_txt_num]) 
    second_max = np.mean(bart_score[llava_txt_num: 2*llava_txt_num])
    third_max = np.mean(bart_score[2* llava_txt_num: 3*llava_txt_num])
    fourth_max = np.mean(bart_score[3*llava_txt_num: 4*llava_txt_num])
    fifth_max = np.mean(bart_score[4*llava_txt_num: ])
    
    if first_max == max(first_max, second_max, third_max, fourth_max, fifth_max):
        correct = correct + 1
    print(correct/num, num)